In [ ]:
! pip install -U pandas==1.5.3

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
match = pd.read_csv("/kaggle/input/ipl-data-set/matches.csv")
delivery = pd.read_csv("/kaggle/input/ipl-data-set/deliveries.csv")
most_run_strikerate = pd.read_csv("/kaggle/input/ipl-data-set/most_runs_average_strikerate.csv")
teams = pd.read_csv("/kaggle/input/ipl-data-set/teams.csv")
home_away = pd.read_csv("/kaggle/input/ipl-data-set/teamwise_home_and_away.csv")

In [ ]:
match.head()

In [ ]:
match.shape

In [ ]:
delivery.head()

In [ ]:
delivery.shape

In [ ]:
total_score_df = delivery.groupby(['match_id', 'inning']).sum()['total_runs'].reset_index()

In [ ]:
total_score_df.head()

In [ ]:
#total score of 1st innings
total_score_df = total_score_df[total_score_df['inning']==1]

In [ ]:
# merging total_score_df with matches
match_df = match.merge(total_score_df[['match_id', 'total_runs']], left_on='id', right_on='match_id')

In [ ]:
match_df['team1'].unique()

In [ ]:
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Kings XI Punjab',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Delhi Capitals'
]

In [ ]:
#changing the name of teams
match_df['team1'] = match_df['team1'].str.replace('Delhi Daredevils', 'Delhi Capitals')
match_df['team2'] = match_df['team2'].str.replace('Delhi Daredevils', 'Delhi Capitals')

match_df['team1'] = match_df['team1'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')

In [ ]:
#keeping only those matches where only existing teams played
match_df = match_df[match_df['team1'].isin(teams)]
match_df = match_df[match_df['team2'].isin(teams)]

In [ ]:
match_df.shape

In [ ]:
#removing dl_applied
match_df = match_df[match_df['dl_applied'] == 0 ]

In [ ]:
#extracting necessary features
match_df = match_df[['match_id', 'city', 'winner', 'total_runs']]

In [ ]:
#merging two Dfs
delivery_df = match_df.merge(delivery, on='match_id')

In [ ]:
delivery_df.head()

In [ ]:
#innings = 2
delivery_df = delivery_df[delivery_df['inning'] == 2]

In [ ]:
delivery_df.shape

In [ ]:
#remaining target after each ball
delivery_df['current_score'] = delivery_df.groupby('match_id').cumsum()['total_runs_y']
delivery_df['runs_left'] = delivery_df['total_runs_x'] - delivery_df['current_score']

In [ ]:
#ball left
delivery_df['balls_left'] = 126 - (delivery_df['over']*6 +delivery_df['ball'])

In [ ]:
delivery_df

In [ ]:
#wickets left
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].fillna("0")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].apply(lambda x:x if x == "0" else "1")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].astype('int')
wickets = delivery_df.groupby('match_id').cumsum()['player_dismissed'].values
delivery_df['wickets'] = 10 - wickets
delivery_df.head()

In [ ]:
delivery_df.tail()

In [ ]:
#crr = runs/overs
delivery_df['crr'] = (delivery_df['current_score']*6)/(120 - delivery_df['balls_left'])

In [ ]:
delivery_df.head()

In [ ]:
#rrr = runs left/balls left
delivery_df['rrr'] = (delivery_df['runs_left']*6)/delivery_df['balls_left']

In [ ]:
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

In [ ]:
delivery_df['result'] = delivery_df.apply(result, axis = 1)

In [ ]:
delivery_df.head()

In [ ]:
final_df = delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr','result']]

In [ ]:
final_df.head()

In [ ]:
final_df = final_df.sample(final_df.shape[0])

In [ ]:
final_df.dropna(inplace=True)

In [ ]:
final_df = final_df[final_df['balls_left'] != 0]

In [ ]:
final_df.head()

In [ ]:
X_train = final_df.iloc[:,:-1]
y_train = final_df.iloc[:, -1]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train,test_size=0.2,random_state=1)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
#handling nominal(categorical) data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

trf = ColumnTransformer([('trf', OneHotEncoder(sparse=False, drop='first'), ['batting_team', 'bowling_team', 'city'])], remainder='passthrough')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.pipeline import Pipeline

In [ ]:
# pipe = Pipeline(steps=[
#     ('step1',trf),
#     ('step2',LogisticRegression(solver='liblinear'))
# ])

In [ ]:
# pipe = Pipeline(steps=[
#     ('step1',trf),
#     ('step2',RandomForestClassifier())
# ])

In [ ]:
# pipe = Pipeline(steps=[
#     ('step1',trf),
#     ('step2',SVC( probability=True))
# ])

In [ ]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',tree.DecisionTreeClassifier())
])

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
pipe.predict_proba(X_test)[10]